# 常用 Metrics 速查（自用笔记）

目标：面试现场快速对齐「任务类型 → 指标 → 解释/取舍」，并能立刻写出 `sklearn` 计算代码。


## 0. 选指标的最小决策树（自检清单）

- 回归：误差量纲是否重要？是否有 outliers / heavy-tail？是否关心极端大错？
- 分类：是否类别不平衡？误报 vs 漏报哪一个更贵？是否需要概率质量（calibration）？
- 排序/选股：更关心排名一致性（IC/Rank IC/Top-k），而不是点预测误差。
- 时间序列：必须避免数据泄露（按时间切分），评估尽量用 walk-forward / expanding window。


## 1. 回归（Regression）常用指标

**核心**：误差的绝对值/平方、鲁棒性、是否可解释为“解释方差”。

| 指标 | sklearn / 公式 | 直观含义 | 特性/注意点 |
|---|---|---|---|
| MAE | `mean_absolute_error` | 平均绝对误差 | 对 outlier 更鲁棒 |
| MSE | `mean_squared_error` | 均方误差 | 强惩罚大错，受 outlier 影响大 |
| RMSE | `sqrt(MSE)` | 均方根误差 | 与目标同量纲，更好解释 |
| MedAE | `median_absolute_error` | 误差中位数 | 极鲁棒；但对尾部不敏感 |
| R² | `r2_score` | 相对基线解释方差 | 不是“预测好坏”的唯一标准；会被分布/切分方式影响 |
| MAPE | `mean_absolute_percentage_error` | 平均绝对百分比误差 | target 接近 0 会爆；对小值过度敏感 |
| SMAPE | 自定义 | 对称百分比误差 | 处理 0 更稳，但仍需 epsilon |


In [ ]:
import numpy as np
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, median_absolute_error,
    r2_score, mean_absolute_percentage_error
)

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def smape(y_true, y_pred, eps=1e-12):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true) + np.abs(y_pred), eps)
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom))

# quick demo (可删)
# y_true = np.array([1,2,3,4])
# y_pred = np.array([1.2,1.7,3.1,10])
# print("MAE", mean_absolute_error(y_true,y_pred))
# print("RMSE", rmse(y_true,y_pred))
# print("R2", r2_score(y_true,y_pred))


### 回归指标的口头解释（常用一句话）

- MAE：平均每个点错多少（对极端点不太敏感）。
- RMSE：更关注大错（大错会被平方放大）。
- MedAE：多数点的典型误差（极端点几乎不影响）。
- R²：相对“预测均值”的基线改善多少（只作补充）。


## 2. 二分类（Binary Classification）常用指标

**核心**：混淆矩阵 + 阈值 + 排序能力 + 概率质量。

| 指标 | sklearn | 依赖阈值 | 适用重点 |
|---|---|---:|---|
| Accuracy | `accuracy_score` | 是 | 类别均衡时可用 |
| Precision | `precision_score` | 是 | 误报贵（False Positive 成本高） |
| Recall | `recall_score` | 是 | 漏报贵（False Negative 成本高） |
| F1 | `f1_score` | 是 | Precision/Recall 折中 |
| ROC-AUC | `roc_auc_score` | 否 | 排序能力（对阈值不敏感） |
| PR-AUC | `average_precision_score` | 否 | 极不平衡时更有意义 |
| LogLoss | `log_loss` | 否 | 概率质量（越小越好） |
| Brier | 自定义 | 否 | 概率校准误差（MSE on prob） |
| MCC | `matthews_corrcoef` | 是 | 不平衡下更稳（综合考虑四格） |
| Balanced Acc | `balanced_accuracy_score` | 是 | 不平衡下的 accuracy 变体 |


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, log_loss,
    matthews_corrcoef, balanced_accuracy_score, confusion_matrix
)

def brier_score(y_true, y_prob):
    y_true = np.asarray(y_true, dtype=float)
    y_prob = np.asarray(y_prob, dtype=float)
    return float(np.mean((y_prob - y_true)**2))

def clf_report_core(y_true, y_pred, y_prob=None):
    out = {}
    out["accuracy"] = float(accuracy_score(y_true, y_pred))
    out["precision"] = float(precision_score(y_true, y_pred, zero_division=0))
    out["recall"] = float(recall_score(y_true, y_pred, zero_division=0))
    out["f1"] = float(f1_score(y_true, y_pred, zero_division=0))
    out["balanced_accuracy"] = float(balanced_accuracy_score(y_true, y_pred))
    out["mcc"] = float(matthews_corrcoef(y_true, y_pred))
    out["confusion_matrix"] = confusion_matrix(y_true, y_pred).tolist()
    if y_prob is not None:
        out["roc_auc"] = float(roc_auc_score(y_true, y_prob))
        out["pr_auc(AP)"] = float(average_precision_score(y_true, y_prob))
        out["log_loss"] = float(log_loss(y_true, y_prob, labels=[0, 1]))
        out["brier"] = brier_score(y_true, y_prob)
    return out

# quick demo (可删)
# y_true = np.array([0,0,1,1,1,0,1])
# y_prob = np.array([0.1,0.4,0.9,0.7,0.2,0.3,0.8])
# y_pred = (y_prob >= 0.5).astype(int)
# print(clf_report_core(y_true, y_pred, y_prob))


### 二分类指标的口头解释（常用一句话）

- Precision：预测为 1 的里面有多少是真的（控制误报）。
- Recall：真的 1 里抓住了多少（控制漏报）。
- F1：Precision 和 Recall 的调和平均（两者都要照顾）。
- ROC-AUC：把正例排在负例前面的概率（排序能力）。
- PR-AUC（Average Precision）：极不平衡时比 ROC-AUC 更敏感，更贴近“抓正例”的效果。
- LogLoss：预测概率是否靠谱（对过度自信惩罚大）。
- Brier：概率预测的均方误差（校准指标）。


## 3. 多分类（Multiclass）常用指标

| 指标 | sklearn | 备注 |
|---|---|---|
| Accuracy | `accuracy_score` | 最常用 |
| Macro-F1 | `f1_score(average="macro")` | 各类等权，关注小类 |
| Weighted-F1 | `f1_score(average="weighted")` | 按样本数加权 |
| Micro-F1 | `f1_score(average="micro")` | 先摊平再算，接近 accuracy |
| LogLoss | `log_loss` | 需要预测概率（softmax） |
| Top-k Acc | `top_k_accuracy_score` | 允许 Top-k 命中 |


In [ ]:
from sklearn.metrics import f1_score, log_loss, top_k_accuracy_score

def multiclass_f1s(y_true, y_pred):
    return {
        "macro_f1": float(f1_score(y_true, y_pred, average="macro", zero_division=0)),
        "weighted_f1": float(f1_score(y_true, y_pred, average="weighted", zero_division=0)),
        "micro_f1": float(f1_score(y_true, y_pred, average="micro", zero_division=0)),
    }

# quick demo (可删)
# y_true = np.array([0,1,2,2,1,0])
# y_pred = np.array([0,2,2,2,1,0])
# print(multiclass_f1s(y_true,y_pred))


## 4. 排序 / 选股 / Recsys（sklearn 不全，但面试常用）

**核心**：很多场景只关心相对排序，而不是绝对数值。

| 指标 | 形式 | 直观含义 |
|---|---|---|
| Spearman Rank IC | 相关系数 | 排名相关（非线性单调也能抓） |
| Pearson IC | 相关系数 | 线性相关 |
| Kendall τ | 一致性 | 排名一致性更严格 |
| Top-k Hit Rate | 命中率 | 选出来的前 k 有多少是好样本 |
| NDCG | 折损增益 | 更强调高位排序正确性 |


In [ ]:
from scipy.stats import spearmanr, pearsonr, kendalltau

def ic_metrics(y_true, y_score):
    y_true = np.asarray(y_true, dtype=float)
    y_score = np.asarray(y_score, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_score)
    y_true = y_true[mask]
    y_score = y_score[mask]
    if len(y_true) < 3:
        return {"pearson_ic": np.nan, "spearman_ic": np.nan, "kendall_tau": np.nan}
    return {
        "pearson_ic": float(pearsonr(y_score, y_true)[0]),
        "spearman_ic": float(spearmanr(y_score, y_true)[0]),
        "kendall_tau": float(kendalltau(y_score, y_true)[0]),
    }

def topk_hit_rate(y_true, y_score, k=10, higher_is_better=True):
    # y_true: 真实目标（可为收益/标签）；y_score: 模型分数
    # 命中定义：预测 top-k 与真实 top-k 的交集占比
    y_true = np.asarray(y_true, dtype=float)
    y_score = np.asarray(y_score, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_score)
    y_true = y_true[mask]
    y_score = y_score[mask]
    if len(y_true) == 0:
        return np.nan
    k = int(min(k, len(y_true)))
    if higher_is_better:
        pred_top = np.argpartition(-y_score, k - 1)[:k]
        true_top = np.argpartition(-y_true, k - 1)[:k]
    else:
        pred_top = np.argpartition(y_score, k - 1)[:k]
        true_top = np.argpartition(y_true, k - 1)[:k]
    return float(len(set(pred_top).intersection(set(true_top))) / k)


## 5. 时间序列/回测相关（常见自定义）

严格意义上很多不在 `sklearn.metrics`，但经常会被问到：

- 方向命中率（sign accuracy）：预测涨跌方向是否对。
- PnL / Sharpe / Sortino / MaxDD：策略层面评价（需要回测框架；面试一般点到为止）。
- 按时间分段的指标稳定性：分月/分季度统计 MAE / IC / hit-rate。


In [ ]:
def sign_accuracy(y_true, y_pred, zero_tie=0.0):
    # 方向命中率：sign(y_pred) == sign(y_true)
    # zero_tie=0.0 表示 sign(0)=0；否则按 >=0 记为正
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    s_true = np.sign(y_true) if zero_tie == 0.0 else np.where(y_true >= 0, 1, -1)
    s_pred = np.sign(y_pred) if zero_tie == 0.0 else np.where(y_pred >= 0, 1, -1)
    return float(np.mean(s_true == s_pred)) if len(s_true) else np.nan


## 6. 指标选择的“默认组合”（快速落地）

### 回归
- 主指标：MAE 或 RMSE（视 outlier/大错惩罚而定）
- 补充：R²（sanity check）

### 二分类（类别不平衡常见）
- 主指标：ROC-AUC 或 PR-AUC（看是否极不平衡）
- 阈值后：Precision / Recall / F1 + confusion matrix
- 概率输出：LogLoss + Brier（看概率质量）

### 多分类
- 主指标：Accuracy + Macro/Weighted F1
- 概率输出：LogLoss
- Top-k 需求：Top-k accuracy

### 排序/选股
- 主指标：Spearman IC / Kendall τ
- 决策层：Top-k hit-rate


## 7. 模板


In [ ]:
# ===== regression =====
# mae = mean_absolute_error(y_true, y_pred)
# rmse_val = rmse(y_true, y_pred)
# r2 = r2_score(y_true, y_pred)

# ===== binary classification =====
# y_prob = model.predict_proba(X)[:, 1]   # 或 decision_function(X) 再 sigmoid
# y_pred = (y_prob >= 0.5).astype(int)
# metrics = clf_report_core(y_true, y_pred, y_prob)

# ===== multiclass =====
# y_prob = model.predict_proba(X)         # shape (n, K)
# y_pred = y_prob.argmax(axis=1)
# f1s = multiclass_f1s(y_true, y_pred)
# ll = log_loss(y_true, y_prob)
# top3 = top_k_accuracy_score(y_true, y_prob, k=3)

# ===== ranking =====
# ic = ic_metrics(y_true, y_score)
# hit10 = topk_hit_rate(y_true, y_score, k=10)


---
备注：这份笔记默认环境为 Python 3.8 + numpy/pandas/scikit-learn；IC 相关用到 `scipy`。
